In [1]:
!sudo pip install -q transformers --upgrade

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [3]:
import torch
from transformers import pipeline

In [14]:


# load model
model = AutoModelForCausalLM.from_pretrained(
    '/data/mistral/query-to-mql/exp-8/oct-30/merged-model',
#     quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, device_map='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mistralai/Mistral-7B-Instruct-v0.1/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


In [15]:
context_data = """{
    "MEASURE": [{"ENTITY": "Discount", "other names": ["discount", "discount rate", "discount value", "deduction"]},
                {"ENTITY": "Purchase Vol", "other names": ["purchase", "purchase value", "purchase model"]},
                {"ENTITY": "Quantity", "other names": ["quantity", "volume"]},
                {"ENTITY": "Sales", "other names": ["sales", "sale"]}],
    "DIMENSION": [{"ENTITY": "Sub-Category", "other names": ["sub-category", "sub category", "categories", "section"]},
                  {"ENTITY": "Segment", "other names": ["segment", "segments", "units", "divisions"]},
                  {"ENTITY": "Parts", "other names": ["parts", "part", "section", "divisions"]},
                  {"ENTITY": "Country", "other names": ["country", "countries"]}],
    "FILTER": [{"ENTITY": "Consumer", "other names": ["consumers", "consumer"], "parent": "Segment"},
               {"ENTITY": "Phone", "other names": ["phone", "phones", "mobile phones"], "parent": "Sub-Category"},
               {"ENTITY": "Binder", "other names": ["binders", "binder"], "parent": "Sub-Category"},
               {"ENTITY": "Corporate", "other names": ["corporates", "corporate"], "parent": "Segment"},
               {"ENTITY": "India", "other names": ["india"], "parent": "Country"},
               {"ENTITY": "Dubai", "other names": ["dubai"], "parent": "Country"}],
    "DERIVED MEASURE": [{"ENTITY": "Ratio",
             "other names": ["ratio", "share", "contribution", "percentage", "proportion", "contributing"]},
            {"ENTITY": "Why", "other names": ["why", "cause of", "reason for", "diagnose"]},
            {"ENTITY": "contribution_to_growth", "other names": ["contribution to growth", "growth", "grown"]},
            {"ENTITY": "kda_transactional", "other names": ["kda", "key drivers", "key driver", "drivers", "driver"]},
            {"ENTITY": "Growth Rate", "other names": ["growth rate", "growth", "grown"]},
            {"ENTITY": "correlation",
             "other names": ["associate", "associated", "association", "associations", "correlate", "correlated",
                             "correlation", "correlations", "relate", "related", "relation", "relations",
                             "relationship",
                             "relationships"]}
            ],
    "DATE VARIABLE": [{"ENTITY": "Order Date", "other names": ["order date", "date", "trend", "time", "when", "mom", "yoy"]}]
    }"""

In [16]:
date_input_data = {
    "start_date": "01/01/2020",
    "end_date": "15/09/2023"
}

In [17]:
query_template = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
"""

In [18]:

query = 'what is sales in 2020'
prompt = query_template.format(context=context_data,
                              date_input=date_input_data,
                              user_query=query)

In [ ]:
t1= time.time()
pipe(prompt)
t2= time.time()
t2-t1

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 801, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


In [7]:
model_name="/data/mistral/query-to-mql/exp-8/oct-30/merged-model"

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                          # add_eos_token=True,
                                          use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [10]:
def predict_template_query_v1(user_query):
    prompt = query_template.format(context=context_data,
                                   user_query=user_query,
                                  date_input=date_input_data)
    _inputs = tokenizer.encode(prompt, return_tensors="pt")
#   outputs = model.generate(input_ids=_inputs.to('cuda'), max_length= 1700, pad_token_id=tokenizer.eos_token_id)
    outputs = pipe(prompt, max_new_tokens=256)
    print(outputs)
#     output = tokenizer.decode(outputs[0])
#     output_new = output.split('[MQL]\n')[1]
#     print(output_new.split('\n[/MQL]')[0])
#     return output_new.split('\n[/MQL]')[0], output
    
#     return output

In [11]:
user_query = 'why sales changed in last 2 weeks of aug 2021'
print('user query: ', user_query)
print('-'*100)
predict_template_query_v1(user_query=user_query)

/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


user query:  why sales changed in last 2 weeks of aug 2021
----------------------------------------------------------------------------------------------------


/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2574: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


KeyboardInterrupt: 

In [12]:
%%time
user_query = 'why sales changed in last 2 weeks of aug 2021'
print('user query: ', user_query)
print('-'*100)
output, raw = predict_template_query_v1(user_query=user_query)
print(eval(output))
print('-'*100)
print('Step 1:' +raw.split('\nStep 1:')[1])

Setting `pad_token_id` to `eos_token_id`:49155 for open-end generation.


user query:  why sales changed in last 2 weeks of aug 2021
----------------------------------------------------------------------------------------------------


ValueError: invalid literal for int() with base 10: 'generated_text'

In [7]:
from refractml import *
from refractml.constants import MLModelFlavours

# # new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

/opt/conda/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [8]:
class ScoreTemplateExample(ScoreBase):
    """
    This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
    """    
    def __init__(self, context_data, date_input_data):
        super().__init__()

        self.model_loaded = None
        
        self.context_data = context_data
        self.date_input_data = date_input_data

        import torch
        from transformers import pipeline

        if self.model_loaded is None:
            print("LLM model loading from data section")
            self.model_loaded = self.prepare_model()
    
    
    def prepare_model(self):
        # load model
        model = AutoModelForCausalLM.from_pretrained(
            '/data/mistral/query-to-mql/exp-8/oct-30/merged-model',
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
        model.config.use_cache = False
        model.config.pretraining_tp = 1

        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", trust_remote_code=True,
                                                  use_fast=False)
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "right"
        
        #pipeline
        pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, device_map='auto')
        return pipe
        
    
    def request_processing_fn(self, request) :
        """
        Processes Request Object -> List[Input data, ..]. It could be:
               A List Mapping of All Value Can Be one of : 
                   - List[ [Feature_Value1, Feature_Value2, ...], [...] ]
                   - List[numpy.array(), numpy.array(), ...]
                   - List[tf.Tensor, tf.Tensor, tf.Tensor, ...]
                   - List[ SingleSample, SingleSample]

        :return: (n_inputs, payload's)

        Warnings:
        1. Do not reshape your final output for single sample here, do it in prediction.
           Else payloads will be invalidated for extraction at raw and extraction level.
        """
        final_payload = []
        raw_payload = request.json["payload"]
        return (1, raw_payload) 
    
    def pre_processing_fn(self,payload):
        return payload

    def prediction_fn(self,
                       model: Any,
                       input_query 
                       ):
        """
                Does the main prediction on pre_processed_input(Single Sample) using supplied model .

                :param model: Supported Model
                :param pre_processed_input: Single Preprocessed Payload
                :return: Prediction Value From the model
                
                Important Notes:
                - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
        """
        prompt_template = """Given the context : {context} and date reference: {date_input}, the query: {user_query}, is converted into below shown structured output.
[MQL]
"""
        text = input_query
        prompt = prompt_template.format(context=self.context_data,
                              date_input=self.date_input_data,
                              user_query=text)
        print("Input being passed to Model ...")
        outputs = self.model_loaded(prompt, max_new_tokens=1800)
        preds = outputs[0]["generated_text"]
        output_new = preds.split('[MQL]\n')[1]
        pred = output_new.split('\n[/MQL]')[0]
        print("prediction is \n: ", pred)
        return pred
        

In [9]:
score_ = ScoreTemplateExample(context_data, date_input_data)
import requests
req = requests.Request()

req.json = {"payload":"How do I sort a list in Python?"}

LLM model loading from data section


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mistralai/Mistral-7B-Instruct-v0.1/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


In [11]:
score_

In [12]:
import time

In [13]:
t3 = time.time()
model_predictions = score_.score(None, req, dry_run=True)
t4 = time.time()
print(t4-t3)

,VALIDATION,COMPONENT,PASSED,SKIPPED
0,"Return Type Must Be Tuple (n_input, payloads)",request_processing_fn,True,False
1,"Tuple Must Be of length Two (n_input, payloads)",request_processing_fn,True,False
2,"* if n_input > 1 payload type must be List (n_input, [np.ndarray, tf.Tensor, etc])",request_processing_fn,True,False


/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Fields Marked Asterisk (*) Can Be Validated On Proper Input 

Input being passed to Model ...


/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:2574: UserWarning: Specified kernel cache directory could not be created! This disables kernel caching. Specified directory is /home/mosaic-ai/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1442.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


KeyboardInterrupt: 